In [ ]:
import os, json, urllib.request, ipykernel

# Try importing for both Jupyter 6.x and 7.x+
try:
    from notebook import notebookapp as serverapp
except ImportError:
    from jupyter_server import serverapp

def get_notebook_dir():
    # Try Jupyter server method (interactive sessions)
    cf = os.path.basename(ipykernel.get_connection_file())
    kid = cf.split('-', 1)[1].split('.')[0]
    for s in serverapp.list_running_servers():
        try:
            with urllib.request.urlopen(s['url'] + 'api/sessions') as resp:
                for sess in json.load(resp):
                    if sess['kernel']['id'] == kid:
                        return os.path.dirname(os.path.join(s['notebook_dir'], sess['notebook']['path']))
        except Exception:
            pass

    # Fallback - just return CWD
    return os.getcwd()


In [ ]:
from pathlib import Path
import os

TRUNCATE_AFTER = "davewalker5.github.io"

def get_project_root_folder():
    # If we're running from the command line, the project root path will be set
    path = os.environ["PROJECT_ROOT"] if "PROJECT_ROOT" in os.environ else None
    if path and os.path.exists(path):
        return path

    # Get the notebook folder and create a Path object from it
    notebook_folder = get_notebook_dir()
    path = Path(notebook_folder)

    # Find the part up to and including the final element after which we truncate
    for parent in path.parents:
        if parent.name == TRUNCATE_AFTER:
            return parent

    # Fallback - just return CWD
    return os.getcwd()



In [ ]:
from pathlib import Path

def get_data_folder_path():
    # Construct the path to the export folder and create it
    project_root = get_project_root_folder()
    data_folder_path = Path(f"{project_root}/_data")
    data_folder_path.mkdir(parents=True, exist_ok=True)

    # Return the path to the folder
    return data_folder_path.absolute()

In [ ]:
import re

def clean_string(string):
    return re.sub("[^0-9a-zA-Z ]+", "", string).replace(" ", "-")